In [36]:
#You will need to be in the correct conda environment: env_camra
import pandas as pd
from openpyxl import Workbook,load_workbook
import json
import os
import re
import sys
#from BeautifulSoup import BeautifulSoup as BS
from bs4 import BeautifulSoup as BS
import urllib3
import requests
import subprocess
import json
import numpy as np

## Add your Assembly location here

In [42]:
BatchAssemblyLocation = "/Users/dmatute/Documents/CAMRA/Bioinformatics/IFAIN/BatchB_250823/Assembly"
#BatchAssemblyLocation = "/Users/dmatute/Documents/CAMRA/Bioinformatics/IFAIN/BatchA_121022/Assembly"

# Location of the sheet that will be populated
excel_location= "/Users/dmatute/Documents/CAMRA/Bioinformatics/Assembly_Metrics.xlsx"

if os.path.isdir(BatchAssemblyLocation) == True and os.path.isfile(excel_location) == True:
    print("Directory and File good")
else:
    print("BAD")

Directory and File good


In [43]:
# Fix location if needed
if BatchAssemblyLocation[-1] != "/": BatchAssemblyLocation = BatchAssemblyLocation + "/"

# Get Batch Name
batch_name = BatchAssemblyLocation.split("/")[-3]

# variable that checks if this batch has many checkm or a single one
have_singlecheckm_file = False

## Functions

In [44]:
# Function to make line in Excel
def append_to_excel(sample_name, sample_data, excel_location, message):
    wb = load_workbook(filename=excel_location)
    ws = wb['Assembly']
    ws.append(sample_data)
    wb.save(filename=excel_location)
    print(message +"    row added to " + os.path.basename(excel_location))

In [45]:
# Function that gets quast for every sample
def get_quast_info (quast_path):
    with open(quast_path, 'r',) as file:
        icount =0
        quast_dict = {}
        for line in file:
            if icount >2:
                value = line.split()[-1]
                description = " ".join(line.split()[:-1])
                quast_dict[description] =value
            icount += 1
    return quast_dict

## Finding MASH file, this file contains the mash output for all samples
if os.path.isfile(BatchAssemblyLocation.replace("Assembly/", "QC/Mash/Mash_Taxa.txt")):
    mash_df = pd.read_csv(BatchAssemblyLocation.replace("Assembly/", "QC/Mash/Mash_Taxa.txt"), sep=",", header=None)
    mash_avaliable = True
else:
    print(batch_name+" Mash_Taxa.txt unavailable")
    mash_avaliable = False


## Finding Lab_assigned_Taxa file
if os.path.isfile(BatchAssemblyLocation.replace("Assembly/", "".join(["Lab_Assigned_Taxa/", batch_name,".xlsx"]))):
    lab_taxa_loc = BatchAssemblyLocation.replace("Assembly/", "".join(["Lab_Assigned_Taxa/", batch_name,".xlsx"]))
    lab_taxa_df = pd.read_excel(lab_taxa_loc)
    lab_taxa_df['sample_name'] =  lab_taxa_df['CAMRA ID'].str.cat(lab_taxa_df['S/N'].astype(str), sep='_S')
    lab_taxa_available = True
else:
    print(batch_name+" Lab_taxa file unavailable")
    lab_taxa_available = False

In [46]:
#ITERATE the SAMPLES
have_singlecheckm_file = False
empty = ""
for sample_name in os.listdir(BatchAssemblyLocation):
    print(">>", sample_name)
    if sample_name[0] != "." :

        message = sample_name
        sample_path = BatchAssemblyLocation + sample_name

        ## QUAST
        if os.path.isfile(sample_path + "/details/" + sample_name+ "_quast_report.txt" ):
            quast_loc = sample_path + "/details/" + sample_name+ "_quast_report.txt" 
            quast_dict = get_quast_info(quast_loc)
            
            quast_longest_contg = float(quast_dict['Largest contig'])
            quast_N50 = float(quast_dict['N50'])
            quast_N90 = float(quast_dict['N90'])
            quast_L50 = float(quast_dict['L50'])
            quast_L90 = float(quast_dict['L90'])
            quast_ASM_SIZE = float(quast_dict['Total length'])
        else:
            message = message +"    quast unavailable"
            quast_longest_contg = empty
            quast_N50 = empty
            quast_N90 = empty
            quast_L50 = empty
            quast_L90 = empty
            quast_ASM_SIZE = empty     
            

        #LAB TAXA
        lab_genusspecies = '' #made for "#Lab expected genome size " validation
        if lab_taxa_available == True:
            find_name = sample_name.split('_')[0]
            lab_output = lab_taxa_df.loc[lab_taxa_df['CAMRA ID'] == find_name].values. flatten().tolist()
            if lab_output!=[]:
                #seperate genus and species form subspecies 
                lab_taxasplit = lab_output[3].split(" ")
                lab_genusspecies = " ".join(lab_taxasplit[:2])
                lab_subspecies = " ".join(lab_taxasplit[2:])
                
                try:
                    ref_command = ["datasets", "summary", "genome", "taxon", lab_genusspecies ,"--reference"]
                    ref_summary = subprocess.run(ref_command, capture_output=True, text=True)
                    ref_summary = ref_summary.stdout.splitlines()
                    ref_summary_dict = json.loads(ref_summary[0])
                    lab_length = float(ref_summary_dict["reports"][0]["assembly_stats"]['total_sequence_length'])
                    asmsize_div_labsize = round(quast_ASM_SIZE/float(lab_length),4)*100
                except:
                    message = message + "    Labtaxa xlsx available, but error finding reference size"
                    asmsize_div_labsize = empty
            else:
                message = message +"    Labtaxa xlsx unavailable"
                lab_genusspecies = empty
                lab_subspecies = empty
                lab_length = empty
                asmsize_div_labsize = empty
            
        else:
            message = message +"    Labtaxa xlsx unavailable"
            lab_genusspecies = empty
            lab_subspecies = empty
            lab_length = empty
            asmsize_div_labsize = empty


        
        ## HTML 
        if os.path.isfile(sample_path + "/" + "AssemblyReport.html"):
            HTML_loc = sample_path + "/" + "AssemblyReport.html"
            with open(HTML_loc, 'r', encoding='utf-8') as file:
                # Read the contents of the file
                html_content = file.read()
                # Parse the HTML using BeautifulSoup
                soup = BS(html_content, 'html.parser')
                div = soup.find_all('div')[-1]
                html_av_read_size = float(div.find_all('td')[-1].text)
                html_numb_reads = float(div.find_all('td') [-2].text)
        else:
            message = message +"    HTML unavailable"
            html_numb_reads = empty
            html_av_read_size = empty


        ## JSON with ASM output metrics
        if os.path.isfile(sample_path + "/details/" + sample_name+ "_run_details.json" ):
            run_loc = sample_path + "/details/" + sample_name+ "_run_details.json" 
            with open(run_loc, 'r') as json_file:
                run_json = json.load(json_file)
                json_read_depth = round(float(run_json['contig_filtering']['average depth (short reads)']),1)
                json_above_thrshld = float(run_json['contig_filtering']['num contigs above thresholds'])
                json_below_thrshld = float(run_json['contig_filtering']['num contigs below thresholds'])
                json_total_size = float(run_json['assembly']['contigs.fasta file size'])
        else:
            message = message +"    run.json unavailable"
            json_read_depth = empty
            json_above_thrshld = empty
            json_below_thrshld = empty
            json_total_size = empty

        
        



        
        #MASH
        if mash_avaliable == True:
            mash_output = mash_df.loc[mash_df[0] == sample_name].values.flatten().tolist()
            if mash_output!=[]:
                #seperate genus and species form subspecies 
                mash_ani = (1-float(mash_output[4]))*100
                mash_taxasplit = mash_output[3].split(" ")
                mash_taxa_id = mash_output[2]
                if mash_ani >= 97.5:
                    mash_genusspecies = " ".join(mash_taxasplit[:2])
                    mash_subspecies = " ".join(mash_taxasplit[2:])
                elif mash_ani < 97.5 and mash_ani >=94 : 
                    mash_genusspecies = " ".join(mash_taxasplit[:2])
                    mash_subspecies = empty
                elif mash_ani < 94 and mash_ani >= 80 :
                    mash_genusspecies = " ".join(mash_taxasplit[:1])
                    mash_subspecies = empty
                elif mash_ani < 80:
                    mash_genusspecies = empty
                    mash_subspecies = empty 
                mash_asm_id = mash_output[1]
            else:
                mash_asm_id = empty
                mash_genusspecies = empty
                mash_subspecies = empty
                mash_ani = empty
                mash_taxa_id = empty
        else:
            message = message +"    Labtaxa xlsx unavailable"
            mash_asm_id = empty
            mash_genusspecies = empty
            mash_subspecies = empty
            mash_ani = empty
            mash_taxa_id = empty

              

        #CheckM
        def checkm_mapping (checkm_dic,sample_name):
            checkm_markerset = checkm_dic[sample_name][2]
            checkm_completness = float(checkm_dic[sample_name][13])
            checkm_contamination = float(checkm_dic[sample_name][14])
            checkm_heterogeneity = float(checkm_dic[sample_name][15])
            return checkm_markerset, checkm_completness, checkm_contamination, checkm_heterogeneity
            
        ## Finding CheckM file, each sample has their own checkm output
        if os.path.isfile(BatchAssemblyLocation.replace("Assembly/", f"QC/CheckM/{sample_name}/checkm_quality_assessment.txt")):
            checkm_file = open(BatchAssemblyLocation.replace("Assembly/", f"QC/CheckM/{sample_name}/checkm_quality_assessment.txt"), "r")
            content=checkm_file.readlines()
            for i in range (0,len(content)):
                if content[i][0:len(sample_name)+2].strip() == sample_name:
                    checkm_metrics = content[i].split()
                    checkm_markerset = checkm_metrics[1]
                    checkm_completness = float(checkm_metrics[12])
                    checkm_contamination = float(checkm_metrics[13])
                    checkm_heterogeneity = float(checkm_metrics[14])
            checkm_file.close()
        #Find checkM file when all the samples in the batch share a single output file
        elif os.path.isfile(BatchAssemblyLocation.replace("Assembly/", f"QC/checkm_quality_assessment.txt")):
            if have_singlecheckm_file == False:
                checkm_file = open(BatchAssemblyLocation.replace("Assembly/", f"QC/checkm_quality_assessment.txt"), "r")
                content=checkm_file.readlines()
                checkm_dic = {}
                for i in range (0,len(content)):
                    if content[i][0:3].strip() == "CCI":
                        checkm_metrics = content[i].split()
                        checkm_sample_name = checkm_metrics[0].split("/")[0]
                        checkm_dic[checkm_sample_name] = checkm_metrics
                have_singlecheckm_file = True
                checkm_file.close()
                checkm_merkerser, checkm_completness, checkm_contamination, checkm_heterogeneity = checkm_mapping(checkm_dic,sample_name)
            else:
                checkm_merkerser, checkm_completness, checkm_contamination, checkm_heterogeneity = checkm_mapping(checkm_dic,sample_name)
        else:
            checkm_markerset = empty
            checkm_completness = empty
            checkm_contamination = empty
            checkm_heterogeneity = empty
            message = message +"    CheckM unavailable"
            

        #Merqury
        merqury_path = BatchAssemblyLocation.replace("Assembly/", f"QC/Merqury/{sample_name}")
        if os.path.isfile(f"{merqury_path}/{sample_name}.qv") and os.path.isfile(f"{merqury_path}/{sample_name}.completeness.stats") :
            merqury_qv_file = open(f"{merqury_path}/{sample_name}.qv", "r")
            merqury_com_file = open(f"{merqury_path}/{sample_name}.completeness.stats","r")
            content_qv=merqury_qv_file.readlines()
            content_com=merqury_com_file.readlines()
            merqury_qv = content_qv[0].split()[3]
            if merqury_qv == "inf":
                merqury_qv = 1000
            else:
                mercury_qv = float(merqury_qv)
            merqury_completeness = float(content_com[0].split()[4])
            merqury_qv_file.close()
            merqury_com_file.close()
        else:
            message = message +"    Merqury unavailable"
            merqury_qv = empty
            merqury_completeness = empty

        
        #Calculations
        if quast_ASM_SIZE != empty and json_total_size!=empty:
            quastASM_div_jsonSize = round(quast_ASM_SIZE/json_total_size,4)*100
        else: quastASM_div_jsonSize = empty
            
        if quast_ASM_SIZE != empty and quast_N90 != empty:
            quastN90_div_quastASM = round(float(quast_dict['N90'])/quast_ASM_SIZE,4)*100
        else:
            quastN90_div_quastASM = empty
        
        
        #MAKE EXCEL ROW
        #if os.path.isfile(sample_path + "/details/" + sample_name+ "_run_details.json" ) and os.path.isfile(sample_path + "/details/" + sample_name+ "_quast_report.txt" ):
        # print(sample_path)
        if os.path.isdir(sample_path) :
 
            sample_data = [sample_name, 
                           batch_name,
                           lab_genusspecies,
                           lab_subspecies, 
                           lab_length,
                           html_numb_reads,
                           html_av_read_size,
                           json_read_depth, 
                           json_above_thrshld,
                           json_below_thrshld, 
                           quast_longest_contg,
                           quast_N50,
                           quast_N90,
                           quast_L50,
                           quast_L90,
                           quast_ASM_SIZE,
                           json_total_size,
                           quastASM_div_jsonSize,
                           asmsize_div_labsize, 
                           quastN90_div_quastASM,
                           mash_taxa_id,
                           mash_asm_id,
                           mash_genusspecies,
                           mash_subspecies,
                           mash_ani,
                           checkm_markerset,
                           checkm_completness,
                           merqury_completeness,
                           checkm_contamination,
                           checkm_heterogeneity,
                           merqury_qv]
        #print(sample_data)



        # SEND TO EXCEL
        append_to_excel(sample_name, sample_data, excel_location, message)  


>> NTC_SEQ_S96
NTC_SEQ_S96    quast unavailable    Labtaxa xlsx unavailable    HTML unavailable    run.json unavailable    CheckM unavailable    Merqury unavailable    row added to Assembly_Metrics.xlsx
>> CCI91_S11
5509643.0 5682322.0
CCI91_S11    row added to Assembly_Metrics.xlsx
>> CCI105_S25
5796485.0 5682322.0
CCI105_S25    row added to Assembly_Metrics.xlsx
>> CCI136_S56
6003371.0 5682322.0
CCI136_S56    row added to Assembly_Metrics.xlsx
>> CCI152_S72
5489201.0 5682322.0
CCI152_S72    row added to Assembly_Metrics.xlsx
>> CCI87_S7
5619095.0 5682322.0
CCI87_S7    row added to Assembly_Metrics.xlsx
>> CCI137_S57
5289872.0 5682322.0
CCI137_S57    row added to Assembly_Metrics.xlsx
>> CCI153_S73
5693088.0 5682322.0
CCI153_S73    row added to Assembly_Metrics.xlsx
>> CCI90_S10
5747089.0 5682322.0
CCI90_S10    row added to Assembly_Metrics.xlsx
>> CCI104_S24
5701098.0 5682322.0
CCI104_S24    row added to Assembly_Metrics.xlsx
>> CCI171_S91
CCI171_S91    Labtaxa xlsx available, but er

In [ ]:
#must be within these vlaues
qc_metric = {"quastASM_div_jsonSize":95,
             "asmsize_div_labsize":[195,95], 

# def pass_or_fail (sample_data)
#     status = "fail"
#     msg = ""
#     if sample_data[15] == "":
#         msg = "Assembly Size = 0"
#         return status
#     else:
#         if sample_data[9]/sample_data[8]*100 > 74:
#             msg = "Bad reads are 74 of the total reas
        
        
        

In [11]:
checkm_location = "/Users/dmatute/Documents/CAMRA/Bioinformatics/IFAIN/CAMRA_BatchA_121022_dir/QC/batch1_combined_checkM_quality_assessment.txt"
batch_name = "CAMRA_BatchA_121022_dir"

#Make directories from checkm files that derrick made
if os.path.isfile(checkm_location):
    checkm_file = open(checkm_location, "r")
    content=checkm_file.readlines()
    for i in range (0,len(content)):
        #print(content[i][0:3].strip(), content[i])
        if content[i][0:3].strip() == "CCI":
            checkm_metrics = content[i].split()
            sample_name = checkm_metrics[0].split("/")[0]
            os.mkdir(f"/Users/dmatute/Documents/CAMRA/Bioinformatics/IFAIN/CAMRA_BatchA_121022_dir/Assembly/{sample_name}")
            # checkm_markerset = checkm_metrics[1]
            # checkm_completness = checkm_metrics[12]
            # checkm_contamination = checkm_metrics[13]
            # checkm_heterogeneity = checkm_metrics[14]
            print(sample_name, checkm_metrics)
        # else:
        #     message = message +"    CheckM unavailable"
    checkm_file.close()

CCI10 ['CCI10/checkm_quality_assessment.txt:', 'CCI10_contigs', 'Klebsiella', '(5)', '29', '1359', '348', '2', '1347', '10', '0', '0', '0', '99.70', '1.26', '20.00']
CCI11 ['CCI11/checkm_quality_assessment.txt:', 'CCI11_contigs', 'Klebsiella', '(5)', '29', '1359', '348', '2', '1347', '10', '0', '0', '0', '99.70', '1.26', '20.00']
CCI12 ['CCI12/checkm_quality_assessment.txt:', 'CCI12_contigs', 'Klebsiella', '(5)', '29', '1359', '348', '68', '1279', '11', '1', '0', '0', '90.41', '0.79', '7.14']
CCI13 ['CCI13/checkm_quality_assessment.txt:', 'CCI13_contigs', 'Klebsiella', '(5)', '29', '1359', '348', '108', '1214', '36', '1', '0', '0', '87.16', '2.46', '0.00']
CCI14 ['CCI14/checkm_quality_assessment.txt:', 'CCI14_contigs', 'Klebsiella', '(5)', '29', '1359', '348', '1', '1354', '4', '0', '0', '0', '99.99', '0.40', '0.00']
CCI15 ['CCI15/checkm_quality_assessment.txt:', 'CCI15_contigs', 'Klebsiella', '(5)', '29', '1359', '348', '97', '1233', '28', '1', '0', '0', '87.74', '2.00', '0.00']
CCI16

In [36]:
a = [1,2,3]
type(a)

list